In [1]:
colab = False

if colab:
    from google.colab import drive
    drive.mount('/content/gdrive')
    !git clone https://github.com/deepmind/pycolab.git
    !git clone https://github.com/nicoladainese96/RelationalModule.git
    !pip install pycolab

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from RelationalModule import ActorCritic, ControlActorCritic
from RelationalModule import train_agent as train
from RelationalModule import utils
from importlib import reload

In [3]:
reload(train)

<module 'RelationalModule.train_agent' from '/home/nicola/Nicola_unipd/MasterThesis/RelationalDeepRL/RelationalModule/train_agent.py'>

In [4]:
game_params = dict(grid_size=12,
                solution_length=[0],
                num_forward = [0], # number of distractors
                num_backward=[0], # just set to 0 for now
                branch_length=1, # length of forward distractors
                max_num_steps = 50
               )

In [5]:
HPs = dict(action_space=4,
           lr=1e-3,
           gamma=0.99,
           TD=True,
           twin=True,
           tau=0.2,
           n_steps=10
           )
if colab:
    HPs['device'] = 'cuda'
else:
    HPs['device'] = 'cpu'

print('device: ', HPs['device'])  

# Relational Agent
agent = ActorCritic.BoxWorldA2C(**HPs)

# Control Agent
control_agent = ControlActorCritic.ControlA2C(**HPs)

device:  cpu
Sequential(
  (0): ExtractEntities(
    (embed): Embedding(117, 3)
    (net): Sequential(
      (0): Conv2d(3, 12, kernel_size=(2, 2), stride=(1, 1))
      (1): ReLU()
      (2): Conv2d(12, 24, kernel_size=(2, 2), stride=(1, 1))
      (3): ReLU()
    )
  )
  (1): RelationalModule(
    (net): Sequential(
      (0): PositionalEncoding(
        (projection): Linear(in_features=26, out_features=256, bias=True)
      )
      (1): AttentionBlock(
        (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (attn): MultiheadAttention(
          (out_proj): Linear(in_features=256, out_features=256, bias=True)
        )
        (ff): PositionwiseFeedForward(
          (w_1): Linear(in_features=256, out_features=64, bias=True)
          (w_2): Linear(in_features=64, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (2): AttentionBlock(
        (norm): LayerNorm((25

In [6]:
# Random Agent

class RandomAgent():
    def __init__(self, n_actions):
        self.n_actions = n_actions
    
    def get_action(self,state, *args, **kwargs):
        a = np.random.choice(self.n_actions)
        log_prob = np.log(1./self.n_actions) # just because it's the standard output of the other agent
        return a, log_prob
    
    def update(self, *args):
        return

rnd_agent = RandomAgent(4)

In [7]:
%%time
results = train.train_boxworld(agent, game_params, n_episodes=100, 
                               max_steps=game_params['max_num_steps'], return_agent=True)
score, asymptotic_score, asymptotic_std, trained_agent, time_profile = results

x.shape (before ExtractEntities):  torch.Size([1, 14, 14])
x.shape (ExtractEntities):  torch.Size([1, 24, 12, 12])
x.shape (After encoding):  torch.Size([1, 26, 12, 12])
x.shape (Before transposing and projection):  torch.Size([1, 26, 144])
x.shape (PositionalEncoding):  torch.Size([144, 1, 256])


/home/nicola/anaconda3/envs/torch/lib/python3.7/site-packages/pycolab/ascii_art.py:318: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  art = np.vstack(np.fromstring(line, dtype=np.uint8) for line in art)


x.shape (RelationalModule):  torch.Size([144, 1, 256])
x.shape (FeaturewiseMaxPool):  torch.Size([1, 256])
x.shape (BoxWorldNet):  torch.Size([1, 256])
x.shape (before ExtractEntities):  torch.Size([1, 14, 14])
x.shape (ExtractEntities):  torch.Size([1, 24, 12, 12])
x.shape (After encoding):  torch.Size([1, 26, 12, 12])
x.shape (Before transposing and projection):  torch.Size([1, 26, 144])
x.shape (PositionalEncoding):  torch.Size([144, 1, 256])
x.shape (RelationalModule):  torch.Size([144, 1, 256])
x.shape (FeaturewiseMaxPool):  torch.Size([1, 256])
x.shape (BoxWorldNet):  torch.Size([1, 256])
x.shape (before ExtractEntities):  torch.Size([1, 14, 14])
x.shape (ExtractEntities):  torch.Size([1, 24, 12, 12])
x.shape (After encoding):  torch.Size([1, 26, 12, 12])
x.shape (Before transposing and projection):  torch.Size([1, 26, 144])
x.shape (PositionalEncoding):  torch.Size([144, 1, 256])
x.shape (RelationalModule):  torch.Size([144, 1, 256])
x.shape (FeaturewiseMaxPool):  torch.Size([1,

x.shape (Before transposing and projection):  torch.Size([1, 26, 144])
x.shape (PositionalEncoding):  torch.Size([144, 1, 256])
x.shape (RelationalModule):  torch.Size([144, 1, 256])
x.shape (FeaturewiseMaxPool):  torch.Size([1, 256])
x.shape (BoxWorldNet):  torch.Size([1, 256])
x.shape (before ExtractEntities):  torch.Size([1, 14, 14])
x.shape (ExtractEntities):  torch.Size([1, 24, 12, 12])
x.shape (After encoding):  torch.Size([1, 26, 12, 12])
x.shape (Before transposing and projection):  torch.Size([1, 26, 144])
x.shape (PositionalEncoding):  torch.Size([144, 1, 256])
x.shape (RelationalModule):  torch.Size([144, 1, 256])
x.shape (FeaturewiseMaxPool):  torch.Size([1, 256])
x.shape (BoxWorldNet):  torch.Size([1, 256])
x.shape (before ExtractEntities):  torch.Size([1, 14, 14])
x.shape (ExtractEntities):  torch.Size([1, 24, 12, 12])
x.shape (After encoding):  torch.Size([1, 26, 12, 12])
x.shape (Before transposing and projection):  torch.Size([1, 26, 144])
x.shape (PositionalEncoding):

x.shape (RelationalModule):  torch.Size([144, 1, 256])
x.shape (FeaturewiseMaxPool):  torch.Size([1, 256])
x.shape (BoxWorldNet):  torch.Size([1, 256])
x.shape (before ExtractEntities):  torch.Size([1, 14, 14])
x.shape (ExtractEntities):  torch.Size([1, 24, 12, 12])
x.shape (After encoding):  torch.Size([1, 26, 12, 12])
x.shape (Before transposing and projection):  torch.Size([1, 26, 144])
x.shape (PositionalEncoding):  torch.Size([144, 1, 256])
x.shape (RelationalModule):  torch.Size([144, 1, 256])
x.shape (FeaturewiseMaxPool):  torch.Size([1, 256])
x.shape (BoxWorldNet):  torch.Size([1, 256])
x.shape (before ExtractEntities):  torch.Size([1, 14, 14])
x.shape (ExtractEntities):  torch.Size([1, 24, 12, 12])
x.shape (After encoding):  torch.Size([1, 26, 12, 12])
x.shape (Before transposing and projection):  torch.Size([1, 26, 144])
x.shape (PositionalEncoding):  torch.Size([144, 1, 256])
x.shape (RelationalModule):  torch.Size([144, 1, 256])
x.shape (FeaturewiseMaxPool):  torch.Size([1,

KeyboardInterrupt: 

On my PC:

 Average time for playing an episode: 1.97428691 <br>
 Average time for updating the agent: 26.46678982

 On Colab notebook:

 Average time for playing an episode:  1.11658633 <br>
 Average time for updating the agent: 18.63836179


With GPU

Average time for playing an episode: 0.43020332 <br>
Average time for updating the agent: 0.64481306


In [8]:
plt.figure(figsize=(8,6))
n_epochs = np.arange(100, len(score))
average_score = np.array([np.mean(score[i:i+100]) for i in range(len(score)-100)])
plt.plot(n_epochs, average_score, alpha=0.9)
plt.title("Performance", fontsize=16)
plt.xlabel("Number of epochs", fontsize=16)
plt.ylabel("Total reward", fontsize=16)
plt.show()

NameError: name 'score' is not defined

<Figure size 576x432 with 0 Axes>

In [ ]:
save = False
keywords = ['control','unboxed_gem',str(len(control_score))+"-episodes"] # example

if colab and save:
    %cd ~
    parent_dir = "/content/gdrive/My Drive/Colab Notebooks/"
    save_dir  = "RelationalTrained/"
    %cd "{parent_dir}"
    !mkdir "{save_dir}"
    ID = utils.save_session(save_dir, keywords, game_params, HPs, score)
    torch.save(trained_agent, save_dir+"agent_"+ID)